# Calculation with splits from paper

## Data loading

In [1]:
import sys
sys.path.insert(0, "../..")
import os
import numpy as np
import scipy.io

In [2]:
# Load diagrams
data_path = "./pd.mat"
data_mat = scipy.io.loadmat(data_path)
data = data_mat["pds"]

print(f"Data shape: {data.shape}")

# y - labels - type of diagram [0-7]
# We transpose to iterate over diagram list
y = []
for label, diagram_list in enumerate(data.T):
    y += len(diagram_list) * [label]
y = np.array(y)

# data.T = 2d array of diagrams as row, column as type
X = data.T.reshape(-1)

Data shape: (50, 7)


In [3]:
# Load splits, these are indexes of points in X, matlab index from 1 (so we do -1)
synth_splits = scipy.io.loadmat('./presplited/synthetic_splits.mat')
test_set = synth_splits['testSets'] - 1
train_set = synth_splits['trainSets'] - 1

tt_pairs = np.array([np.array((train, test)) for train, test in zip(train_set, test_set)])


## First step: Model hyperparameter optimization

In [6]:
import joblib
import pandas as pd
import sklearn

from preprocessing import *
from persistence_bow import *
from visualization import *
from persistence_fv import *
from experiments.utils import *

from sklearn.cluster import KMeans, MiniBatchKMeans
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.model_selection import train_test_split, cross_validate, cross_val_score, StratifiedKFold, GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC, LinearSVC
from sklearn.mixture import GaussianMixture

from gudhi.representations.kernel_methods import SlicedWassersteinKernel
from gudhi.representations.vector_methods import PersistenceImage
from gudhi.representations.metrics import BottleneckDistance


In [7]:
#Helper function, constructs final pipeline and returns girdsearch for it
def make_final_grid(estimator,
                    param_grid,
                    kernel="linear",
                    cv=tt_pairs,
                    **kwargs):
    new_param_grid = {f"Model__{name}" : values for name, values in param_grid.items()}
    new_param_grid["Predictor__C"] = [0.1, 1, 10]
    
    final_pipeline = Pipeline([
        ("Model", estimator),
        ("Predictor", SVC(kernel=kernel, max_iter=1e6))
    ])
    
    return GridSearchCV(final_pipeline, new_param_grid, cv = cv, **kwargs)

In [8]:
#PBoW gridsearch
pbow_gridsearch = make_final_grid(
    estimator = PersistenceBow(KMeans(7, n_init=1, max_iter=100, random_state=42),
                              sampler=RandomPDSampler(2500, random_state=42)),
    param_grid = {
        "cluster__n_clusters" : np.arange(10, 150, 15),
        "sampler__max_points" : np.arange(1000, 10000, 2000),
        "sampler__weight_function" : [const, linear]
    },
    n_jobs = -1
)

#SPBoW gridsearch
spbow_gridsearch = make_final_grid(
    estimator = StablePersistenceBow(GaussianMixture(covariance_type="diag", random_state=42),
                              sampler=RandomPDSampler(2500, random_state=42)),
    param_grid = {
        "mixture__n_components" : np.arange(10, 200, 15),
        "sampler__max_points" : np.arange(1000, 13000, 2000),
        "sampler__weight_function" : [const, linear]
    },
    n_jobs = -1
)

#PBoW gridsearch with grid sampler
pbow_grid_gridsearch = make_final_grid(
    estimator = PersistenceBow(KMeans(7, n_init=1, max_iter=100, random_state=42),
                              sampler=GridPDSampler((10,10), 100, random_state=42)),
    param_grid = {
        "cluster__n_clusters" : np.arange(10, 150, 15),
        "sampler__max_points" : [10, 50, 100, 200, 500],
        "sampler__grid_shape" : [(5,5), (10,10), (15,15), (20,20), (40,40)],
        "sampler__weight_function" : [const, linear]
    },
    n_jobs = -1
)

#SPBoW gridsearch with grid sampler
spbow_grid_gridsearch = make_final_grid(
    estimator = StablePersistenceBow(GaussianMixture(covariance_type="diag", random_state=42),
                              sampler=GridPDSampler((10,10),100, random_state=42)),
    param_grid = {
        "mixture__n_components" : np.arange(10, 200, 15),
        "sampler__max_points" : [10, 50, 100, 200, 500],
        "sampler__grid_shape" : [(5,5), (10,10), (15,15), (20,20), (40,40)],
        "sampler__weight_function" : [const, linear]
    },
    n_jobs = -1
)


# Fisher_vector 
fisher_vector_gridsearch = make_final_grid(
    PersistenceFV(sampler=RandomPDSampler(random_state=42)), 
    param_grid = {
        "gmm_clusters_number" : np.arange(10, 200, 15),
        "sampler__max_points" : np.arange(1000, 13000, 2000),
        "sampler__weight_function" : [const, linear]},
    n_jobs= - 1)

# Fisher_vector with grid sampler
fisher_vector_grid_gridsearch = make_final_grid(
    PersistenceFV(sampler=GridPDSampler((10,10), 100,random_state=42)), 
    param_grid = {
        "gmm_clusters_number" : np.arange(10, 200, 15),
        "sampler__max_points" : [10, 50, 100, 200, 500],
        "sampler__grid_shape" : [(5,5), (10,10), (15,15), (20,20), (40,40)],
        "sampler__weight_function" : [const, linear]},
    n_jobs= - 1)



models_to_test = {
    "PBoW" : pbow_gridsearch, 
    "PBoW_Grid" : pbow_grid_gridsearch,
    "SPboW" : spbow_gridsearch,
    "SPBoW_Grid" : spbow_grid_gridsearch,
    "FisherVector": fisher_vector_gridsearch,
    "FisherVector_Grid": fisher_vector_grid_gridsearch
}

In [12]:
#Hyperparameter optimization
for name, grid in models_to_test.items():
    print(name)
    grid_path = f"grid/{name}.dill"
    
    out = load(grid_path)
    if out:
        print("Loaded from file")
        models_to_test[name] = out
    else:
        grid.verbose = 10
        grid.fit(X, y)
        save(grid, grid_path)

PBoW
Loaded from file
PBoW_Grid
Loaded from file
SPboW
Loaded from file
SPBoW_Grid
Loaded from file
FisherVector
Loaded from file
FisherVector_Grid
Loaded from file


In [14]:
#Grid computation
grid_path = f"./grid/"
cv_path = f"./cv/"

for filename in os.listdir(grid_path):
    name = os.path.splitext(filename)[0]
    grid = load(os.path.join(grid_path, filename))
    results = load(os.path.join(cv_path, filename))
    
    if not results:
        print("Computing", name)
        model = grid.best_estimator_
        results = cross_validate(model, X, y, cv=tt_pairs, n_jobs=-1, verbose=10)
        save(results, os.path.join(cv_path, f"{name}.dill"))
    else:
        print(f"Loaded results: {name}")


Loaded results: PBoW_Grid
Loaded results: SPboW
Loaded results: SPBoW_Grid
Loaded results: PBoW
Loaded results: FisherVector
Loaded results: FisherVector_Grid


In [15]:
base_path = "./cv"
for filename in os.listdir(base_path):
    path = os.path.join(base_path, filename)
    name = os.path.splitext(filename)[0]
    result = load(path)
    print(name, "Test set score:\n", result["test_score"].mean(), "\n")


SPBoW_Grid Test set score:
 0.9800000000000001 

FisherVector_Grid Test set score:
 0.9714285714285715 

PBoW Test set score:
 0.9714285714285715 

FisherVector Test set score:
 0.9657142857142856 

SPboW Test set score:
 0.9771428571428572 

PBoW_Grid Test set score:
 0.9742857142857144 

